# <center>Practical HPC</center>
<p>
## <center>Robert Bjornson</center> 
<p>
## <center><i>Yale Center for Research Computing</i></center>
<p>
## <center>March 2018</center>

## What is the Yale Center for Research Computing?


- Independent center under the Provost's office
- Created to support your research computing needs
- Focus is on high performance computing and storage
- ~15 staff, including applications specialists and system engineers
- Available to consult with and educate users
- Manage compute clusters and support users
- Located at 160 St. Ronan st, at the corner of Edwards and St. Ronan
- http://research.computing.yale.edu

## Overview
- Understanding the HPC environment
- Understanding your programs behavior
- Getting the right answer
- Getting your answer faster

# Assumptions
- Have logged in to cluster and run simple jobs
- understand basics of Slurm


## What is HPC?
- running jobs remotely 
- using more resources than you have locally
- access to large shared resources
- running in parallel, or many sequential jobs

# Typical Cluster

![alt text](cluster.png "Title")

# Typical Cluster

![alt text](yalecluster.jpg "Title")

## Typical Cluster (Yale, or elsewhere)
- Several hundred nodes
- Each node has 10s of cpus -> Thousands of total cpus
- Each node has hundreds of GB RAM
- Multiple PB of shared storage
- Fast network connecting nodes and networks



## Cluster Resources
- Nodes and CPUs
- Node Memory
- Job Runtime
- Disk Storage
- GPUs
- etc

Nodes, cpus, memory, runtime, GPUs are requested via Slurm options, and enforced by OS.

Storage limits enforced by filesystem quotas.



# Quick Review of Slurm allocations

$ sbatch [options] batch.sh

$ srun --pty [options] bash

option | example | comment
-------|--------|---
-p _partition_ | -p general | partitions(s) to run on
-c _cores_ | -c 20 | cores/task on single node
-n _tasks_ | -n 4 | mpi progs only
-N _nodes_ | -N 10 | "never" useful
-t _time_  | -t 7-, -t 3:00 | job killed if exceeded
--mem=_mem_ | --mem=16g | ditto
--mem-per-cpu=_mem_ | --mem-per-cpu=8g | ditto
-J _name_ | -J myjob | name job
--gres | --gres=gpu:p100:2 | request gpus


## What is "Running Efficiently"?
- All allocated cores are running near 100%
- memory requested and used are commensurate
- File and Network I/O are "reasonable"

## Typical overall user limits (vary by cluster)
- cpus (few hundred)
- memory (5 GB * cpus)
- storage
 - home: hundreds GB, 500k files
 - project: few TB (shared among group), 5M files
 - scratch: many TB (shared among group), 5M files
 - file counts matter!


## Storage Quotas
- varies somewhat by cluster, see cluster doc
- examples:
 - (farnam) /ysm-gpfs/bin/my_quota.sh, group_quota.sh
 - (ruddle) /ycga-gpfs/bin/my_quota.sh, group_quota.sh
 - (grace) /gpfs/apps/bin/groupquota.sh 
 - (omeaga-next?)

## Nodes/CPUs
- only request multiple cpus and/or nodes if your program can use them
- simply requesting more nodes or cpus will not make things run faster!
- only request GPU or bigmem nodes if you need them


# Monitoring your job's resource requirements
```
squeue -u netid -l (your jobs)
squeue -j jobid -l (particular job)
scontrol show job jobid (during job)
sacct -j jobid -l (after job finishes)
```
Note that the default output for squeue and sacct are not idea.  Put in .bashrc:
```
export SACCT_FORMAT="JobID%-20,JobName,User,Partition,NodeList,Elapsed,State,ExitCode,MaxRSS, AllocTRES%32"
export SQUEUE_FORMAT="%.16i %.12P %.12j %.8u %.2t %.12M %.12l %24R %.4D %.4C %m %8b %8f"
```



In [ ]:
NOTE run through entire example (sbatch, scontrol, sacct /usr/bin/time -a )

# Checking cpu and memory utilization for running jobs
- use ```squeue -u netid``` to determine where your job is running
- ssh to those nodes, especially 2nd and on, and run ```top -u netid```
- look at %CPU and RES columns
- should see ~%100 cpu for each allocated core
EXAMPLE HERE

# Thinking about memory requirements
- default 5GB/allocated cpu on our clusters
- strictly enforced; jobs exceeding limit killed
- you can request custom memory per node or core with sbatch or srun:
```
--mem=6g
--mem-per-cpu=6g
```

# Determining memory requirements for completed jobs

1. Before run begins:
$ /usr/bin/time -a _prog args_

2. After run completes, determine actual usage:
$ sacct -j jobid
EXAMPLE


# Slurm sacct

sacct -o 'JobID,MaxRSS,MaxVMSize' -j _jobid_

or 

Configure sacct format:

 export SACCT_FORMAT=JobID%-20,JobName,User,Partition,NodeList,Elapsed,State,ExitCode,MaxRSS, AllocTRES%32
 
 sacct -j _jobid_




# Remora
https://github.com/TACC/remora

module load REMORA
remora prog args ...

This will create a directory: remora_jobid

Copy (rsync) to local computer, open remora_summary.html with browser 



# Finding Compute Resources

To get overall sense:
```
sinfo -p general
```
To see completely idle nodes, by core count:

```
$ sinfo -p general -e -t IDLE -o "%P %.5a %c %.10l %.6D %.6t %N"
PARTITION AVAIL CPUS  TIMELIMIT  NODES  STATE NODELIST
general*    up 8 30-00:00:0     35   idle c06n[10-16],c07n[01-14,16],c08n[01-06,08-14]
general*    up 16 30-00:00:0     23   idle c10n[13-16],c11n[01-16],c12n[09-11]
```

Hint, use alias:
```
alias findidle='sinfo -p general -e -t IDLE -o "%P %.5a %c %.10l %.6D %.6t %N"'
```

# Fairshare scheduling
- Groups and users with heavy recent usage (last 30-45 days) have lower priority


# Using Scavenge Partition
- Compute nodes in other partions are available via scavenge partition
- sbatch -p scavenge ...
- separate per user limits apply
- works best for short jobs, dSQ/array jobs, or jobs that checkpoint

# Special Nodes

# Large Memory Nodes
- We have some compute nodes with 512GB-1.5TB of RAM
- Reserved for applications with large memory needs. Please be considerate.
- Separate slurm partition: bigmem

Typical allocation: 
```
srun/sbatch -p bigmem --mem=1500g ...
```

# GPU Nodes
- Some applications have been ported to GPUs with impressive performance improvement
- Gpu nodes have conventional cpus with multiple cores, and 1-4 GPUs.  
- To use GPUs, you must:
 - request node(s) with GPUs
 - request the type and number of GPUs 

Typical allocation:
```
srun/sbatch -p gpu -c 20 --gres=gpu:1080ti:4 ...
```
Note that partition names, types and number of GPUs vary by cluster.


# Parallelism

- Sbatch can allocate multiple cores and nodes, but the script runs on one core on one node sequentially.

- Simply allocating more nodes or cores DOES NOT make jobs faster.

- How do we use multiple cores to increase speed?


- Two classes of parallelism:
 - Lots of independent sequential jobs
 - Single job parallelized (somehow)
 

- Some options:
 - Submit many batch jobs simultaneously (not good)
 - Use job arrays, or dSQ (much better)
 - Submit a parallel version of your program (great if you have one)



# Job Arrays

- Useful when you have many nearly identical, independent jobs to run
- Starts many identical copies of your script, distinguished by a task id.

Submit jobs like this:
```
sbatch --array=1-100 ...
```
Inside your batch script this environment variable to do something different in each task:
```
./mycommand -i input.${SLURM_ARRAY_TASK_ID} \
    -o output.${SLURM_ARRAY_TASK_ID}
```

A few nice features of job arrays:
- only one job to keep track of
- easy to start or cancel entire set
- time limits apply to each task, not overall job
- your allocation can grow and shrink as conditions change
- when using scavenge partition, tasks are killed, but job persists


# dSQ (aka Dead Simple Queue)
- built on job arrays.  Same nice features, but easier to use
- more flexible; tasks can be different from one another
- reporting and error recovery built in


# Using dSQ


- Create file containing list of commands to run (jobs.txt)
```
prog arg1 arg2 -o job1.out
prog arg1 arg2 -o job2.out
...
```
- Create launch script
```
module load dSQ
dSQ --taskfile jobs.txt [slurm args] > run.sh
```

slurm args can specify partion, timelimit, memory, etc. in the usual way.

- Submit launch script
```
sbatch run.sh
```

For more info, see <http://research.computing.yale.edu/support/hpc/user-guide/dead-simple-queue>



# dSQ Reporting
- When dSQ job is finished, you'll see a file `job_<jobid>_status.tsv`
- Generate report:
```
$ dSQAutopsy jobs.txt job_<jobid>_status.tsv > failedjobs.txt
Autopsy Task Report:
9 succeeded
1 failed
0 didn't run.
```

- If any jobs failed, failedjobs.txt will contain those jobs





# Some ways to run in parallel
- R: multicore
- Python: multiprocessing
- C: threads

# Namd example
- molecular dynamics simulation
- STMV virus 1,066,628 atoms, 500 time steps


![alt text](Examples\Namd\namd.png "Title")


# Namd Performance on ? Cluster

   |startup (s)|simulate|total|step|step s/u
---|-------|--------|-----|----|----
1 cpu|26|?|~5400|11.6|1.0
20 cpu|64|270|334|0.54|21
20 cpu+4 gpu|64|32|96|.064|181

# Namd Performance on Grace

   |startup (s)|simulate|total|step|step s/u
---|-------|--------|-----|----|----
1 node, 1 cpu|14|5833|5847|11.6|1.0
1 node, 28 cpu|13|218|231|0.44|26.3
1 node, 20 cpu+4 K80 gpus|12|32|44|.056|207
9 nodes, 40 cpus|29.5|212|242|.424|27.4

in /home/fas/lsprog/rdb9/repos/PracticalHPC/Examples/Namd/stmv
- gpus slurm-9139008.out
- mpi slurm-9152709.out
- 28 core MP slurm-9139007.out
- 1 core slurm-9140416.out

In [ ]:
#show 
- namd on 1 cpu
- namd on 1 cpu asking for more
- namd on several nodes
memory required (while running, sacct)


# R Multicore

Many R packages have parallelism built in: e.g. bootstrapping (boot)

```
cores=Sys.getenv("SLURM_CPUS_ON_NODE")
boot(data=trees, statistic=volume_estimate, R=50000, parallel="multicore", ncpus=cores)
```

# What's actually going on?
R uses lapply to apply a function to an array of values.  
multicore uses parlapply to do the same thing in parallel with almost no change to code
